In [1]:
import pandas as pd
import numpy as np
import glob
import os
import tiledb
tiledb.libtiledb.version()

(2, 3, 4)

In [2]:
import findspark
findspark.init()

# import SparkContext and SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [3]:
conf = SparkConf().setMaster("local").setAppName("spark test")
sc = SparkContext(conf = conf)
ss = SparkSession.builder.master('local').appName('spark test')\
    .getOrCreate()
ss

# Create TileDB from pySpark Dataframe

## Read json data into pySpark dataframe

After I downgrade spark 3.1.1 to 2.4.4 and python 3.7.9 to 3.6. There are no confilict for pyspark api, `tiledb-spark-0.5.3.jar`, with native pySpark json/csv reader and writer. I can sucessfully read the json file and write csv. So my previous problem seems to be version related, or how the spark is discovered.

In [4]:
# Load data
study_index = ss.read.json('data/study_index_sample.json')
# study_index = ss.read.format("org.apache.spark.sql.execution.datasources.json.JsonFileFormat")\
#                 .load('data/study_index_sample.json')

In [5]:
study_index.count()

97

In [6]:
study_index.show(5)

+--------------------+--------------------+------------+-------+---------+-------------+--------------+-------------+-------------+----------+--------------+--------------------+----------+--------------------+-------------+--------------------+
|    ancestry_initial|ancestry_replication|has_sumstats|n_cases|n_initial|n_replication|num_assoc_loci|         pmid|   pub_author|  pub_date|   pub_journal|           pub_title|  study_id|      trait_category|   trait_efos|      trait_reported|
+--------------------+--------------------+------------+-------+---------+-------------+--------------+-------------+-------------+----------+--------------+--------------------+----------+--------------------+-------------+--------------------+
|      [European=146]|                null|       false|     96|      146|         null|             1|PMID:15761122|     Klein RJ|2005-03-10|       Science|Complement factor...|GCST000001|      Nervous system|[EFO_0001365]|Age-related macul...|
|[Asian unspecif

LOOK at the SCHEMA. These are:
- `String Arrays` for attributes: ancestry_initial, ancestry_replication, trait_efos 
- `Boolean` for attribute: has_sumstats


In [7]:
study_index.printSchema()

root
 |-- ancestry_initial: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ancestry_replication: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- has_sumstats: boolean (nullable = true)
 |-- n_cases: long (nullable = true)
 |-- n_initial: long (nullable = true)
 |-- n_replication: long (nullable = true)
 |-- num_assoc_loci: long (nullable = true)
 |-- pmid: string (nullable = true)
 |-- pub_author: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- pub_journal: string (nullable = true)
 |-- pub_title: string (nullable = true)
 |-- study_id: string (nullable = true)
 |-- trait_category: string (nullable = true)
 |-- trait_efos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- trait_reported: string (nullable = true)



## Create TileDB Array from pySpark Dataframe

In the documentation, TileDB can be create directly from Spark Dataframe directly. But there seems to be some version compatability issues. And also there are much less options and flexibilities, such as the empty values. Therefore, here we write Spark dataframe to csv and then create TileDB from CSV files.

NullPointer exception while writing your data. It seems that there is a bug in our implementation. As a temporary solution and to help you get unstuck, try converting all types to String. 

In [8]:
# from pyspark.sql.functions import concat, concat_ws, lit, col, trim
# from pyspark.sql.types import StringType
# for each_col in study_index.columns:
#     study_index = study_index.withColumn(each_col,col(each_col).cast(StringType()))


In [15]:
from pyspark.sql.functions import concat, concat_ws, lit, col, trim
from pyspark.sql.types import StringType

study_index= study_index.withColumn("trait_efos",col("trait_efos").cast(StringType()))
study_index= study_index.withColumn("ancestry_replication",col("ancestry_replication").cast(StringType()))
study_index= study_index.withColumn("ancestry_initial",col("ancestry_initial").cast(StringType()))
study_index = study_index.withColumn("has_sumstats",col("has_sumstats").cast(StringType()))


In [16]:
study_index.printSchema()

root
 |-- ancestry_initial: string (nullable = true)
 |-- ancestry_replication: string (nullable = true)
 |-- has_sumstats: string (nullable = true)
 |-- n_cases: long (nullable = true)
 |-- n_initial: long (nullable = true)
 |-- n_replication: long (nullable = true)
 |-- num_assoc_loci: long (nullable = true)
 |-- pmid: string (nullable = true)
 |-- pub_author: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- pub_journal: string (nullable = true)
 |-- pub_title: string (nullable = true)
 |-- study_id: string (nullable = true)
 |-- trait_category: string (nullable = true)
 |-- trait_efos: string (nullable = true)
 |-- trait_reported: string (nullable = true)



In [17]:
study_index.write\
  .format("io.tiledb.spark")\
  .option("uri", "study_index_tldb")\
  .option('schema.dim.0.name', 'study_id')\
  .save()

Py4JJavaError: An error occurred while calling o207.save.
: org.apache.spark.SparkException: Writing job aborted.
	at org.apache.spark.sql.execution.datasources.v2.WriteToDataSourceV2Exec.doExecute(WriteToDataSourceV2Exec.scala:92)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:80)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:80)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(Da

# Create from pandas

In [137]:
df = pd.read_json('data/study_index_sample.json', lines=True)

In [119]:
df = df.set_index('study_id')

In [87]:
df.to_csv('study_index.csv')

In [111]:
df['pmid'] = df['pmid'].astype('|S')

In [125]:
df.astype({'pmid':'|S'})


,pmid,pub_date,pub_journal,pub_title,pub_author,trait_reported,ancestry_initial,n_initial,n_cases,trait_category,num_assoc_loci,has_sumstats,trait_efos,ancestry_replication,n_replication
study_id,,,,,,,,,,,,,,,
GCST000001,b'PMID:15761122',2005-03-10,Science,Complement factor H polymorphism in age-relate...,Klein RJ,Age-related macular degeneration,[European=146],146,96.0,Nervous system,1,False,[EFO_0001365],NaN,NaN
GCST000002,b'PMID:16252231',2005-09-09,Am J Hum Genet,High-resolution whole-genome association study...,Maraganore DM,Parkinson's disease,"[Asian unspecified=1, European=744, Other=141]",886,NaN,Nervous system,0,False,[EFO_0002508],"[Other=122, Asian unspecified=1, European=541]",664.0
GCST000003,b'PMID:16614226',2006-04-14,Science,A common genetic variant is associated with ad...,Herbert A,Obesity,[NR=694],694,NaN,Disease,0,False,[EFO_0001073],"[European=10527, African American or Afro-Cari...",11622.0
GCST000004,b'PMID:16648850',2006-04-30,Nat Genet,A common genetic variant in the NOS1 regulator...,Arking DE,QT interval,[European=200],200,NaN,Cardiovascular measurement,1,False,[EFO_0004682],[European=4851],4851.0
GCST000005,b'PMID:17052657',2006-09-28,Lancet Neurol,Genome-wide genotyping in Parkinson's disease ...,Fung HC,Parkinson's disease,[European=537],537,267.0,Nervous system,0,False,[EFO_0002508],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCST000080_3,b'PMID:17903294',2007-09-19,BMC Med Genet,Genome-wide association and linkage analyses o...,Yang Q,Hemostatic factors and hematological phenotype...,[European=1062],1062,NaN,Hematological measurement,0,False,[EFO_0004527],NaN,NaN
GCST000080_4,b'PMID:17903294',2007-09-19,BMC Med Genet,Genome-wide association and linkage analyses o...,Yang Q,Hemostatic factors and hematological phenotype...,[European=1062],1062,NaN,Protein measurement,0,False,[EFO_0004792],NaN,NaN
GCST000080_5,b'PMID:17903294',2007-09-19,BMC Med Genet,Genome-wide association and linkage analyses o...,Yang Q,Hemostatic factors and hematological phenotype...,[European=1062],1062,NaN,Hematological measurement,0,False,[EFO_0004305],NaN,NaN


In [112]:
cols = [col for col in df.columns if col not in ['pub_date', 'n_initial','n_cases','num_assoc_loci','n_replication']]
cols

['pmid',
 'pub_journal',
 'pub_title',
 'pub_author',
 'trait_reported',
 'ancestry_initial',
 'trait_category',
 'has_sumstats',
 'trait_efos',
 'ancestry_replication']

In [132]:
df[cols] = df[cols].fillna('').astype(str)

In [133]:
df.dtypes

pmid                     object
pub_date                 object
pub_journal              object
pub_title                object
pub_author               object
trait_reported           object
ancestry_initial         object
n_initial                 int64
n_cases                 float64
trait_category           object
num_assoc_loci            int64
has_sumstats             object
trait_efos               object
ancestry_replication     object
n_replication           float64
dtype: object

In [72]:
# df = df.fillna(0).astype({"pmid": str, "pub_date": 'datetime64','has_sumstats':str})
# df.dtypes

In [134]:
import shutil
if tiledb.array_exists("study_index_tldb", isdense=False, issparse=False):
    shutil.rmtree('study_index_tldb')
    
tiledb.from_pandas("study_index_tldb", df.fillna(''), chunksize = 10**6,
                fillna={"trait_efos":'', 'ancestry_replication':'','ancestry_initial':''})

NotImplementedError: mixed inferred dtype not supported

# Create TileDB from CSV

## Write Spark dataframe to CSV

Need to convert array data type to string in order to be able to save to csv files.

In [ ]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType

# def array_to_string(my_list):
#     if my_list:
#         return ';'.join([str(elem) for elem in my_list])
#     else:
#         return ''

# array_to_string_udf = udf(array_to_string, StringType())

# study_index_ps = study_index.withColumn('trait_efos_str', array_to_string_udf(study_index["trait_efos"]))
# study_index_ps = study_index_ps.withColumn('ancestry_replication_str', array_to_string_udf(study_index["ancestry_replication"]))
# study_index_ps = study_index_ps.withColumn('ancestry_initial_str', array_to_string_udf(study_index["ancestry_initial"]))

In [ ]:
# study_index = study_index.drop("trait_efos").drop('ancestry_replication').drop('ancestry_initial')

In [18]:
study_index.show(5)

+--------------------+--------------------+------------+-------+---------+-------------+--------------+-------------+-------------+----------+--------------+--------------------+----------+--------------------+-------------+--------------------+
|    ancestry_initial|ancestry_replication|has_sumstats|n_cases|n_initial|n_replication|num_assoc_loci|         pmid|   pub_author|  pub_date|   pub_journal|           pub_title|  study_id|      trait_category|   trait_efos|      trait_reported|
+--------------------+--------------------+------------+-------+---------+-------------+--------------+-------------+-------------+----------+--------------+--------------------+----------+--------------------+-------------+--------------------+
|      [European=146]|                null|       false|     96|      146|         null|             1|PMID:15761122|     Klein RJ|2005-03-10|       Science|Complement factor...|GCST000001|      Nervous system|[EFO_0001365]|Age-related macul...|
|[Asian unspecif

In [20]:
study_index.write.csv("study_index_csv",header=True,mode='overwrite')
# study_index.write.format('com.databricks.spark.csv') \
#             .save("study_index_csv",header=True,mode='overwrite')

## Create TileDB Array from CSV

For attributes that you need to use with QueryCondition, the data type needs to be specified as "ascii". This can be done by passing a dictionary with type overrides to the `column_types` keyword argument of `from_csv`. 

"ascii" requires TileDB-Py version 0.10.3 or 0.10.4. For earlier versions, you could also use {"study_id": bytes} -- this will translate into a non-string type in the future, however. "ascii" will ensure that the underlying array schema is marked as text (TILEDB_ASCII).

In [25]:
import glob
study_index_csv = sorted(glob.glob('study_index_csv/*.csv'))
study_index_csv

['study_index_csv\\part-00000-b47b2890-815d-4e81-adc6-e321b4eb9e1e-c000.csv']

In [135]:
import shutil
if tiledb.array_exists("study_index_tldb", isdense=False, issparse=False):
    shutil.rmtree('study_index_tldb')
    
tiledb.from_csv("study_index_tldb", 'study_index.csv', chunksize = 10**6,
                fillna={"trait_efos":'', 'ancestry_replication':'','ancestry_initial':''})

In [77]:
import shutil
if tiledb.array_exists("study_index_tldb", isdense=False, issparse=False):
    shutil.rmtree('study_index_tldb')

tiledb.from_csv("study_index_tldb", 
                study_index_csv,
                index_dims = ['study_id'],
                chunksize = 10**6,
                fillna={"trait_efos":'', 'ancestry_replication':'','ancestry_initial':''},
                column_types = {"trait_category": bytes},
                varlen_types={'trait_category'}

               )

TileDBError: Multiple input CSV files requires a 'chunksize' argument

In [31]:
schema = tiledb.ArraySchema.load('study_index_tldb')
schema

ArraySchema(
  domain=Domain(*[
    Dim(name='study_id', domain=(None, None), tile='None', dtype='|S0'),
  ]),
  attrs=[
    Attr(name='ancestry_initial', dtype='<U0', var=True, nullable=False),
    Attr(name='ancestry_replication', dtype='<U0', var=True, nullable=False),
    Attr(name='has_sumstats', dtype='uint8', var=False, nullable=False),
    Attr(name='n_cases', dtype='float64', var=False, nullable=False),
    Attr(name='n_initial', dtype='int64', var=False, nullable=False),
    Attr(name='n_replication', dtype='float64', var=False, nullable=False),
    Attr(name='num_assoc_loci', dtype='int64', var=False, nullable=False),
    Attr(name='pmid', dtype='<U0', var=True, nullable=False),
    Attr(name='pub_author', dtype='<U0', var=True, nullable=False),
    Attr(name='pub_date', dtype='<U0', var=True, nullable=False),
    Attr(name='pub_journal', dtype='<U0', var=True, nullable=False),
    Attr(name='pub_title', dtype='<U0', var=True, nullable=False),
    Attr(name='trait_category',